In [1]:
import pandas as pd
import numpy as np
import csv
import os
import json
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import PunktSentenceTokenizer #ai that can be used to train on parts of speech
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
path ="metadata.csv" 
df = pd.read_csv(path, encoding = 'utf-8')

In [2]:
#directories = ["biorxiv_medrxiv","comm_use_subset","custom_license","noncomm_use_subset"]
directories = ["biorxiv_medrxiv"]
json_array = []
for d in directories:
    for file in os.listdir(f"{d}/{d}"):
        #print(file)
        file_path = f"{d}/{d}/{file}"
        j = json.load(open(file_path, "rb"))
        json_array.append(j)
#finds json files and saves to an array

In [3]:
len(json_array)

885

In [4]:
#search_keys 
for key in json_array[0]:
    print(key)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [5]:
#start with 0 out of 885 lol I need to automate this but good for first test run
raw_abstract = []
raw_metadata = []
raw_bodytext = []
raw_bib_entries = []
raw_ref = []
raw_back_mater = []
for i in range(len(json_array)):
    raw_abstract.append(json_array[i]['abstract'])
    raw_metadata .append(json_array[i]['metadata'])
    raw_bodytext.append(json_array[i]['body_text'])
    raw_bib_entries.append(json_array[i]['bib_entries'])
    raw_ref.append(json_array[i]['ref_entries'])
    raw_back_mater.append(json_array[i]['back_matter'])

In [6]:
body_full_text = []
for i in range(len(raw_bodytext)):
    for key in raw_bodytext[i]:
        body_full_text.append(key['text'] + '\n\n')

In [7]:
##------------------------------------------------------------##
#raw_bodytext
#body_full_text = []
#for key in raw_bodytext:
    #body_full_text.append(key['text'] + '\n\n')
#    print(key['text'])

In [8]:
len(body_full_text)#there are 20 paragraphs here

30181

In [9]:
#needs more structuring.
#Tasks as per Kaggle:- what is known about transmission/incubation and environmental stability?
###Points to consider::--- Extract meaning(NLP?) Terms displayed in Tasks may be called something else in the dataset/research papers...(WordNet)
#body_full_text

In [10]:
#Range of incubation periods for the disease in humans (and how this varies across age and health status)
#and how long individuals are contagious, even after recovery.
#body_full_text[2]
#tokenize sentences

In [42]:
paragraphs = []
for i in range(len(body_full_text)):
    paragraphs.append(sent_tokenize(body_full_text[i]))

In [52]:
len(paragraphs[0]) #how many sentences are in a paragraph

2

In [54]:
#first step, remove stop words
len(paragraphs)

30181

## Stop words

In [63]:
stop_words = set(stopwords.words("english"))
#print(stop_words)

In [56]:
i = 0
while i < len(paragraphs):
    temp = len(paragraphs[i])
    j = 0
    while j < temp:
        temp_word = paragraphs[i][j]
        temp_array = []
        temp_filtered = []
        temp_array = word_tokenize(temp_word)
        for word in temp_array:
            if word not in stop_words:
                temp_filtered.append(word)
        paragraphs[i][j] = temp_filtered
        j = j+ 1
    i = i+ 1
        

In [64]:
#paragraphs[1][3]

## Similarity Indexing

We are looking for range in incubation periods in humans only and how it varies across age and health status.
Step 1, generate a "similarity index" for each paragraph. The higher the similarity index, the more likely we are to find what we are looking for.

In [68]:
similarity_dictionary = {}

In [87]:
#target_words = ["incubation","humans","age","health"]
target_words = ["incubation","humans"]
target_length = len(target_words)
print (target_length)

4


Wordnet will be used to acquire these similarities. For my dictionary, i only need associated paragraph/sentence and attatched overall similarity in the form of a percentage. Start with base similarity search.

In [83]:
#data
#percentage
sentense_score_array = []
i = 0
while i < len(paragraphs):
    temp = len(paragraphs[i])
    j = 0
    while j < temp:
        temp_word = paragraphs[i][j]
        k = 0
        temp_score = 0
        raw_word_score = 0
        for word in temp_word:
            while k < target_length:
                try:
                    w1=wordnet.synsets(word)[0]
                    w2=wordnet.synsets(target_words[k])[0]
                    percentage = (w1.wup_similarity(w2))*100
                except:
                    percentage = 0
                percentage+=percentage
                k = k + 1
            raw_word_score += (percentage/target_length)
        try:
            raw_sentense_score = raw_word_score/len(temp_word)
        except:
            raw_sentense_score = 0
        #print("sentense_score: ",raw_sentense_score)
        sentense_score_array.append(raw_sentense_score)
        j = j+ 1
    i = i+ 1  

In [88]:
#sentense_score_array

In [89]:
import statistics
print(statistics.mean(sentense_score_array))
max(sentense_score_array)

13.104143245669704


96.55172413793109

In [17]:
import re
#remember..should be raw strings!
x = "incubation is 523 days and 12 weeks"
pattern = re.compile(r'\d+ (days|day|weeks|week|months|month|years|year)')
matches = pattern.finditer(x)

for match in matches:
    print(match)
    
print(x[5:7]) #string slicing

<re.Match object; span=(14, 22), match='523 days'>
<re.Match object; span=(27, 35), match='12 weeks'>
at
